In [3]:
import tensorflow as tf
import larq as lq
import larq_compute_engine as lce

In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

In [3]:
# All quantized layers except the first will use the same options
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")

model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
model.add(lq.layers.QuantConv2D(32, (3, 3),
                                kernel_quantizer="ste_sign",
                                kernel_constraint="weight_clip",
                                use_bias=False,
                                input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

In [4]:
lq.models.summary(model)

+sequential stats------------------------------------------------------------------------------------------+
| Layer                  Input prec.           Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs  32-bit MACs |
|                              (bit)                        x 1       x 1    (kB)                          |
+----------------------------------------------------------------------------------------------------------+
| quant_conv2d                     -  (-1, 26, 26, 32)      288         0    0.04           0       194688 |
| max_pooling2d                    -  (-1, 13, 13, 32)        0         0       0           0            0 |
| batch_normalization              -  (-1, 13, 13, 32)        0        64    0.25           0            0 |
| quant_conv2d_1                   1  (-1, 11, 11, 64)    18432         0    2.25     2230272            0 |
| max_pooling2d_1                  -    (-1, 5, 5, 64)        0         0       0           0            0 |
| batch_normalizati

In [5]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=64, epochs=6)

test_loss, test_acc = model.evaluate(test_images, test_labels)

Epoch 1/6
938/938 [==============================] - 29s 31ms/step - loss: 0.6455 - accuracy: 0.9096
Epoch 2/6
938/938 [==============================] - 28s 30ms/step - loss: 0.4736 - accuracy: 0.9622
Epoch 3/6
938/938 [==============================] - 28s 30ms/step - loss: 0.4481 - accuracy: 0.9690
Epoch 4/6
938/938 [==============================] - 28s 29ms/step - loss: 0.4356 - accuracy: 0.9731
Epoch 5/6
938/938 [==============================] - 28s 29ms/step - loss: 0.4291 - accuracy: 0.9755
Epoch 6/6
313/313 [==============================] - 2s 6ms/step - loss: 0.4460 - accuracy: 0.9699


In [6]:
print(f"Test accuracy {test_acc * 100:.2f} %")

Test accuracy 96.99 %


In [8]:
# Convert our Keras model to a TFLite flatbuffer file
with open("./tflite-models/mnist.tflite", "wb") as flatbuffer_file:
    flatbuffer_bytes = lce.convert_keras_model(model)
    flatbuffer_file.write(flatbuffer_bytes)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /var/folders/3x/f8xkz0ld28s8ty74_v94rnp40000gn/T/tmpmfqquklo/assets


Repeat experiments but with neural networks that have all continuous precision layers. 

In [12]:
model = tf.keras.models.Sequential()

# In the first layer we only quantize the weights and not the input
model.add(lq.layers.QuantConv2D(32, (3, 3),
                                use_bias=False,
                                input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

In [13]:
lq.models.summary(model)

+sequential_3 stats--------------------------------------------------------------------+
| Layer                   Input prec.           Outputs  # 32-bit  Memory  32-bit MACs |
|                               (bit)                         x 1    (kB)              |
+--------------------------------------------------------------------------------------+
| quant_conv2d_9                    -  (-1, 26, 26, 32)       288    1.12       194688 |
| max_pooling2d_6                   -  (-1, 13, 13, 32)         0       0            0 |
| batch_normalization_14            -  (-1, 13, 13, 32)        64    0.25            0 |
| quant_conv2d_10                   -  (-1, 11, 11, 64)     18432   72.00      2230272 |
| max_pooling2d_7                   -    (-1, 5, 5, 64)         0       0            0 |
| batch_normalization_15            -    (-1, 5, 5, 64)       128    0.50            0 |
| quant_conv2d_11                   -    (-1, 3, 3, 64)     36864  144.00       331776 |
| batch_normalization

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, batch_size=64, epochs=6)

test_loss, test_acc = model.evaluate(test_images, test_labels)

Epoch 1/6
938/938 [==============================] - 26s 27ms/step - loss: 0.5609 - accuracy: 0.9551
Epoch 2/6
938/938 [==============================] - 27s 28ms/step - loss: 0.4871 - accuracy: 0.9743
Epoch 3/6
938/938 [==============================] - 26s 28ms/step - loss: 0.4706 - accuracy: 0.9792
Epoch 4/6
478/938 [==============>...............] - ETA: 13s - loss: 0.4633 - accuracy: 0.9808

In [ ]:
print(f"Test accuracy {test_acc * 100:.2f} %")

In [ ]:
# Convert our Keras model to a TFLite flatbuffer file
with open("./tflite-models/mnist_full_precision.tflite", "wb") as flatbuffer_file:
    flatbuffer_bytes = lce.convert_keras_model(model)
    flatbuffer_file.write(flatbuffer_bytes)